# HOW2 

The following notebook will walk you through some basic python that will help you create triangles and graphs.
Make sure you **run all of the code blocks in the order they appear** in this notebook, as they can depend on each other.
I have tried to make the variable names as self explanatory as possible.

If a line starts with #, then it is a comment and will explain the code it surrounds.


## Imports
In the cell below, all that is happening is we are importing the packages we will be using in this tutorial 

In [ ]:
%matplotlib inline
import matplotlib
#from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Where we have written e.g. ```import matplotlib.pyplot as plt``` all this means is we are importing it and renaming it under an alias we choose. This just makes our lives easier as when we want to call the function, we can just type ```plt``` instead of
the entire ```matplotlib.pyplot```

The ```%matplotlib inline``` allows us to view the graphs in this notebook

# Gross Premiums Received

In the section below, we will create triangles and graphs relating to the 'Gross Premiums Received' from the data in the data.csv file.

## Reading from a .csv File

In the cell below, we are reading the data in the claims.csv file into a DataFrame (using the ```pd.read_csv(FILENAME)``` function). A DataFrame is basically just a spreadsheet. You can name the DataFrame anything, in this example, we have chosen the variable name df.

We don't want commas to be polluting our numbers so, to remove them, use ```df = df.replace(',','', regex = True)``` where the ```regex = True``` option will also replace commas inside substrings

I'm also going to convert the "Processing Month" column to type string, so it is subscriptable. This will be useful in a later exercise

The ```display(df)``` command will print a pretty table displaying the DataFrame.

In [ ]:
df = pd.read_csv('claims.csv')

df.replace(',','', regex = True, inplace=True)

# here we just change the pandas settings so we can see all the rows in the tables. You can change it if you would like. 
# see what happens when you change None to 4 (make sure to change it back afterwards for the rest of the tutorial)
pd.set_option("display.max_rows", None)

# I am converting this column to type string so i can do string slicing on it later
df['Processing Month'] = df['Processing Month'].astype('str')

#I am converting the Gross Premiums Received, Gross Paid Claims, Gross Incurred Claims columns to integer
df[['Gross Premiums Received', 'Gross Paid Claims', 'Gross Incurred Claims']] = df[['Gross Premiums Received', 'Gross Paid Claims', 'Gross Incurred Claims']].astype('int')

display(df)

# an alternative to display() is print(tabulate(df, headers = 'keys', tablefmt = 'pretty')), if you want to use it, uncomment
# it in the imports and install it on your machine

# Displaying a triangle

We will be using a built in pandas function to create our triangles. 

# Gross Premiums Received

We want to create a pivot table with the Underwriting YoA against the Development period. 

## Finding the development periods
The development period values are held in another column in our DataFrame, but as an extra challenge, lets instead infer it from the processing months.
Lets find the development period using vector calculations. The formula we want to use is: (Processing year - Underwriting YoA) *12 + Processing month

We can start by splitting Processing Month into two columns: Processing Year and Processing Month.

In [ ]:
# creating a DataFrame with the 3 columns we will need
vec_gross_prems_piv= df[['Underwriting YoA','Processing Month','Gross Premiums Received']].copy()

# making'Processing Month' into 2 columns: 'Processing Month' and 'Processing Year'
# using a built in function str to string slice every value in column aand then converting it to integers again
# so we can use the values in our calculations
vec_gross_prems_piv['Processing Year'] = vec_gross_prems_piv['Processing Month'].str[:4].astype('int')
vec_gross_prems_piv['Processing Month'] = vec_gross_prems_piv['Processing Month'].str[4:6].astype('int')

#using our formula to create a list containing the development periods
dev_period = []
dev_period = (vec_gross_prems_piv['Processing Year']-vec_gross_prems_piv['Underwriting YoA'])*12 + vec_gross_prems_piv['Processing Month']
vec_gross_prems_piv['Development period'] = dev_period

## Create a Triangle using Pivot Tables

We can now create the triangle from our DataFrame using the [pd.pivot_table function](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html). We will also insert a "Grand Total" row and column.

the ```margins = True``` inserts a Totals row and column using the ```margins_name = 'Grand Total'``` as the titles.

In [ ]:
table_gross_prems = pd.pivot_table(vec_gross_prems_piv, values='Gross Premiums Received', index=['Underwriting YoA'],
                    columns=['Development period'],aggfunc= {"Gross Premiums Received":np.sum} ,margins = True, 
                    margins_name = 'Grand Total')

display(table_gross_prems)

## Displaying Graphs

In this next section we will display the Gross Premiums as a graph similar to this (we won't let it drop down):
![](Gross_premiums_graph_example.png)

### Plot the graph

If you use ```df.plot.line()``` it plots all the numerical columns as separate lines. We can't just straight away plot the pivot table using .plot.line() because it will plot the wrong indexes, we want to transpose it first. If you want further explanation, read the [documentation for .plot.line()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.line.html).    

We will use our pivot table, but we will first remove the "Grand Total" from the column and row.

In [ ]:
# remove the Totals columns 
table_gross_prems.drop('Grand Total', inplace=True, axis = 1)
table_gross_prems.drop('Grand Total', inplace=True, axis = 0)

# plot the graph
ax = table_gross_prems.transpose().plot.line()


# "print" the graph
plt.show()

### Add the axis labels, etc

You may want to clean up the graph a bit by adding axes labels and a title.  
It might also be worth changing the settings so all your graphs appear bigger.  
You may also want to be able to save the graph as an image.  
If you are struggling with this, or want to know more, the [matplotlib documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html) will help, and so will the [pandas.DataFrame.plot documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html).

In [ ]:
# makes our graph a little bit bigger
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize']=(10, 7)

# plot our DataFrame  
ax = table_gross_prems.transpose().plot.line()

# create axes labels and title the graph
ax.set(xlabel='Time elapsed (months)', ylabel='Gross Premiums Received', title='Gross Premiums Received')

#If you want to display the y axis as $50M, you can create a function to do this (we can then call this later on)
def yaxisFormatting(x, y):
    return str('${:.0f}'.format(x/1000000) +'M')
                                      
plt.gca().yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(yaxisFormatting))

# If you want to display the y axis in the form $50,000,000, comment out the line above and uncomment the line below
#plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('${x:,.0f}'))

# this will save the graph as an image, search in the directory you are in and open it
plt.savefig('gross_prem_graph.png')

# if you wanted to plot multiple graphs in the same figure, this is also possible 
# if you wanted to display the graphs as next to each other, change fig, axes = plt.subplots(1,2)
fig1, axes = plt.subplots(2)

#setting a figure title
fig1.suptitle('DISPLAYING MULTIPLE GRAPHS:')

# im just plotting the same graph twice. If you want to plot different ones, change a table_gross_prems.transpose() to the name of a different DataFrame
table_gross_prems.transpose().plot(ax= axes[0]) 
table_gross_prems.transpose().plot(ax= axes[1]) 

# "print" the graphs
plt.show()

# Gross Incurred Claims

Lets repeat the process for Gross Incurred Claims. We already worked out the development period, so we can just append it to the DataFrame.

In [ ]:
# create a DataFrame with the colums we need
gross_inc_piv= df[['Underwriting YoA','Processing Month','Gross Incurred Claims']].copy()

# create a new column containing the development periods
gross_inc_piv['Development period'] = dev_period

# create our pivot table
table_gross_inc = pd.pivot_table(gross_inc_piv, values='Gross Incurred Claims', index=['Underwriting YoA'],
                  columns=['Development period'],aggfunc= {"Gross Incurred Claims":np.sum} ,margins = True, 
                    margins_name = 'Grand Total')

display(table_gross_inc)

## Displaying the graph

In [ ]:
# remove the Totals column and row
table_gross_inc.drop('Grand Total', inplace=True, axis = 1)
table_gross_inc.drop('Grand Total', inplace=True, axis = 0)

#transpose the table so it plots the graph how we would like
ax = table_gross_inc.transpose().plot.line()

#change axis labels
ax.set(xlabel='Development period', ylabel='Gross Incurred Claims', title='Gross Incurred Claims')
plt.gca().yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(yaxisFormatting))
plt.show()

# Gross Paid Claims
 
We will now do the same for the Gross Paid Claims

In [ ]:
# create our dataframe 
gross_paid_piv= df[['Underwriting YoA','Processing Month','Gross Paid Claims']].copy()
gross_paid_piv['Development period'] = dev_period

# create our pivot table
table_gross_paid = pd.pivot_table(gross_paid_piv, values='Gross Paid Claims', index=['Underwriting YoA'],
                   columns=['Development period'],aggfunc= {"Gross Paid Claims":np.sum} ,margins = True, 
                    margins_name = 'Grand Total')

display(table_gross_paid)

## Displaying the graph

In [ ]:
# remove the totals column and row 
table_gross_paid.drop('Grand Total', inplace=True, axis = 1)
table_gross_paid.drop('Grand Total', inplace=True, axis = 0)


# transpose the table so it plots the graph how we would like
ax = table_gross_paid.transpose().plot.line()

#change axis labels
ax.set(xlabel='Development period', ylabel='Gross Paid Claims', title='Gross Paid Claims')
plt.gca().yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(yaxisFormatting))
plt.show()

# What to do if your data isn't cumulative

We have the file where the data is not cumulative: claims2.csv. In this section, we will adjust the data until it is.

In [ ]:
df2 = pd.read_csv('claims2.csv')

df2.dropna(inplace = True, thresh = 2)
df2.replace(',','', regex = True, inplace=True)
#convert the Gross Premiums Received, Gross Paid Claims, Gross Incurred Claims columns to int
df2[['Gross Premiums Received', 'Gross Paid Claims', 'Gross Incurred Claims']] = df[['Gross Premiums Received', 
                                                                                     'Gross Paid Claims', 
                                                                                     'Gross Incurred Claims']].astype('int')

pd.set_option("display.max_rows", None)
display(df2)

You can make a column cumulative by using the function cumsum(). We want to be more specific and want each year to have an individual cumulative sum. We can achieve this by using the groupby() function.

In [ ]:
# you can see what the groupby() function does by uncommenting the line below
#display(df2.groupby(['Underwriting YoA'])['Gross Premiums Received'].count())

df2['Gross Premiums Received'] = df2.groupby(['Underwriting YoA'])['Gross Premiums Received'].cumsum()
df2['Gross Paid Claims'] = df2.groupby(['Underwriting YoA'])['Gross Paid Claims'].cumsum()
df2['Gross Incurred Claims'] = df2.groupby(['Underwriting YoA'])['Gross Incurred Claims'].cumsum()
display(df2)

In [ ]:
df3 = vec_gross_prems_piv.copy()

#remove unecessary columns
df3.drop(['Processing Year', 'Processing Month'], axis =1, inplace = True)

#we need every 3rd month, so will remove rows if the development period is not divisible by 3 
df3 = df3[(df3['Development period'] % 3 == 0)]

# a dictionary containing the number of years that have each development period
year_list = df3.groupby(['Development period'])['Underwriting YoA'].apply(list).to_dict()

#a list containing the names of the different years in the list i.e. [2013, 2014, 2015, 2016, 2017]
year_name_list= df3['Underwriting YoA'].unique()

# a list of the unqique development periods in the column i.e. [3,6,9,12 ...]
devprd_list = df3['Development period'].unique().tolist()

# keeping the values in lists as these will become the columns of our dataframe
development_factor_list = []
numerators = []
denominators = []

numerator = 0
# set the initial denominator to the value of the first columns sum 
denominator = df3.loc[df3['Development period']==devprd_list[0],'Gross Premiums Received'].sum()

numerators.append(np.nan)
denominators.append(np.nan)

for i, devprd in enumerate(devprd_list): 
    # we want to ignore the first value as the development factor relies on the datapoint before it
    if devprd != devprd_list[0]:
        # the numerator is equal to the sum of all the rows where the development period is the same
        numerator = df3.loc[df3['Development period']==devprd,'Gross Premiums Received'].sum()
        # if the dev periods dont have data for corresponding years
        if year_list[devprd] !=  year_list[devprd_list[i-1]]:
            # a set of the common years between the development periods
            common_years = set(year_list[devprd]) & set(year_list[devprd_list[i-1]])
            numerator = 0
            denominator = 0
            for j in common_years:
                #set the appropriate numerator and denominator for the datapoints in the common set of years
                numerator = numerator + df3.loc[((df3['Development period']==devprd) &(df3['Underwriting YoA']==j))]['Gross Premiums Received'].tolist()[0]
                denominator = denominator + df3.loc[((df3['Development period']==devprd_list[i-1]) &(df3['Underwriting YoA']==j))]['Gross Premiums Received'].tolist()[0]
        development_factor_list.append(numerator/denominator)
        numerators.append(numerator)
        denominators.append(denominator)
        denominator = numerator

# we insert NaN for the first devprd because we have no denominator value for it -> so it wont have a development factor
development_factor_list.insert(0, np.nan)
df3new = pd.DataFrame()
    
df3new['Development period'] = devprd_list
df3new['Number of data points'] = [len(k) for k in year_list.values()]
df3new['Numerator'] = numerators
df3new['Denominator'] = denominators
df3new['Development factor'] = development_factor_list
display(df3new)

# Finding development percentages


In [ ]:
# overall development factor product
overall_devprod = df3new['Development factor'].product()

# get the cumulative product for each value then divide by the value of the overall product
dev_pcnt = df3new['Development factor'].cumprod().div(overall_devprod).tolist()

# append the new column onto the DataFrame
df3new['Development percentages'] = dev_pcnt

display(df3new)


# Development percentages with picked values

Since these are picked manually, we will just select an index after which each value will be set to 100% (1). In this case, we have chosen every value after index 11.

In [ ]:
# copy development percentages
dev_pcnt_with_pick = df3new['Development percentages'].copy().tolist()

# append to the DataFrame
df3new['Development percentages with pick'] = dev_pcnt_with_pick

# now recalculate the cumulative product in a seperate column so we can choose the values we want
df3new['temp cumulative product'] = df3new['Development factor'].cumprod().tolist()
df3new['Development percentages with pick'] = df3new['Development factor'].cumprod().div(df3new['temp cumulative product'][11])

# remove the temporary column
df3new.drop(['temp cumulative product'], axis = 1, inplace =True)

# replace all values after a certain index (we have picked 11) with 1
df3new.loc[11:, 'Development percentages with pick'] = 1

# also add the value for the first column manually, we want this to be dev pct / dev factor of the next row
df3new.loc[0,'Development percentages with pick'] =  df3new.loc[1,'Development percentages']/df3new.loc[1,'Development factor']

display(df3new)



# new table 

We will use the [last_valid_index()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.last_valid_index.html) function to find the latest development period. 

This is also a [good stack overflow post](https://stackoverflow.com/questions/40583482/getting-last-non-na-value-across-rows-in-a-pandas-dataframe) to understand whats happening with indexing the last non-NaN value.

In [ ]:
# instantiate the new DataFrame
df4 = pd.DataFrame()

# inserting the years as a column
df4['Underwriting YoA'] = year_name_list

# adapted from the referenced stack overflow post
# this function returns the last valid non na value in the referenced row
def getPremsReceived(x):
    return x[x.last_valid_index()]

# Find the latest development period for each year (from our old pivot table)
latest_dev_prd = table_gross_prems.apply(pd.Series.last_valid_index, axis=1).tolist()
df4['Latest development period'] = latest_dev_prd

#.apply(axis = 1) means apply the function to each row
df4['Premiums Received'] = table_gross_prems.apply(getPremsReceived, axis=1).tolist()

devPcnt = []

#find the corresponding premiums developed % (with pick) from the last DataFrame we made 
for i in latest_dev_prd:
    # converting the value to float before adding into the df4 
    devPcnt.append(float(df3new.loc[((df3new['Development period']==i))]['Development percentages with pick'].tolist()[0]))

# add the list we just made into a new column in our DataFrame
df4['% premiums developed'] = devPcnt
    
df4['Ultimate premiums'] = df4['Premiums Received'] / df4['% premiums developed']



display(df4)

# Table with Gross Premiums Received / Ultimate premiums



In [ ]:
# take just the data we need from the other columns
df5 = df3.copy()

# A copy of df4 containing the columns we need, so we can add the ultimate premiums onto our new table
df4copy = pd.DataFrame()
df4copy[['Underwriting YoA', 'Ultimate premiums']] = df4[['Underwriting YoA', 'Ultimate premiums']].copy()

# merge the two DataFrames. This essentially will just add the Ultimate premiums values into another column depending on
# the value in the Underwriting YoA column
df5 = df5.merge(df4copy, on = 'Underwriting YoA', how='left')

# Another column that is the result of Gross Premiums Received/ Ultimate premiums
df5['Percentage gross prems'] = df5['Gross Premiums Received']/df5['Ultimate premiums']

# we dont want the gross premiums received column anymore, so remove it
df5.drop(['Gross Premiums Received'], axis =1, inplace =True) 

# convert the result to a pivot table
pivdf5 = pd.pivot_table(df5, values ='Percentage gross prems', index = df5['Underwriting YoA'], columns = df5['Development period'])

display(pivdf5)

# Plotting the Expectations Graph

This time, we will add an extra line that is not already in the dataframe to represent the selected pattern. We will make this line thicker and dashed so it stands out.

In [ ]:
#plot the graph, and make the linewidth of the lines other than the selected pattern smaller
ax = pivdf5.transpose().plot.line(linewidth=0.75)

#change axis labels
ax.set(xlabel='Development period')

# we want to also display the selected pattern from df3new 
selected_pattern = df3new['Development percentages with pick'].tolist()

# add the extra line for our selected pattern. k-- makes it black and dashed so its easier to differentiate between the lines
plt.plot(devprd_list, selected_pattern, 'k--', label = 'Selected Pattern')

# a new formatting function we can use to display the axis scale as percentages
def ypercentformat(x,y):
    return str('{:.2f}'.format(x*100) + '%')

# since we added an extra line we have to manually add the legend
ax.legend()
    
plt.gca().yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(ypercentformat))

plt.show()

# Create a function that outputs both graphs!

We will create multiple subfunctions as it is better to make your code more modular. This is because it makes debugging easier and it aids readability.

Pretty much none of the code is brand new, but some has been slightly adapted to be more general. An example of this would be df_latestDevPrd['% ' + column_name +  ' developed'] instead of df_latestDevPrd['% premiums developed]. This is so it can be relevant for all the other possible column names.


In [ ]:
#If you want to display the y axis as $50M, you can create a function to do this (we can then call this later on)
def yaxisFormatting(x, y):
    return str('${:.0f}'.format(x/1000000) +'M')

# a new formatting function we can use to display the axis scale as percentages
def ypercentformat(x,y):
    return str('{:.2f}'.format(x*100) + '%')

# this function returns the last valid non na value in the referenced row
def getLastNonNA(x):
    return x[x.last_valid_index()]

# makes our graphs a little bit bigger
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize']=(10, 7)

#Gross Incurred Claims is a special case. we will import a table into this variable so we can have access to BF Prior / Initial LR
tabledf = pd.read_csv('2017q3.csv')
#remove rows with only NaN values
tabledf.dropna(axis = 0, how = 'all', inplace = True)


def plotEverything(df, column_name):
    listOfParams = createPivTable(df, column_name)
    plotBothGraphs(listOfParams)

# This function creates the pivot table 
def createPivTable(df, column_name):
    df_piv_data= df[['Underwriting YoA','Processing Month', column_name]].copy()
    df_piv_data['Processing Year'] = df_piv_data['Processing Month'].str[:4].astype('int')
    df_piv_data['Processing Month'] = df_piv_data['Processing Month'].str[4:6].astype('int')
    df_piv_data['Development period'] = (df_piv_data['Processing Year']-df_piv_data['Underwriting YoA'])*12 + df_piv_data['Processing Month']
   
    # making a copy of the premiums data so it can be used in case the column name is 'Gross Incurred Claims'
    df_prem_piv_data = df_piv_data.copy()
    df_prem_piv_data.rename({column_name:'Gross Premiums Received'}, inplace = True)
    df_prem_piv_data['Gross Premiums Received'] = df['Gross Premiums Received']
    prem_piv_table = pd.pivot_table(df_prem_piv_data, values='Gross Premiums Received', index=['Underwriting YoA'],
                    columns=['Development period'])
    piv_table = pd.pivot_table(df_piv_data, values=column_name, index=['Underwriting YoA'],
                    columns=['Development period'])
    listOfParams = buildExpectationsTables(df, column_name, df_piv_data, piv_table, prem_piv_table)
    return listOfParams
    
# This function starts the (longer) process of building the expectations graph
def buildExpectationsTables(df, column_name, df_piv_data, piv_table, prem_piv_table):
    df1 = df_piv_data.copy()
    df1.drop(['Processing Year', 'Processing Month'], axis =1, inplace = True)
    df1 = df1[(df1['Development period'] % 3 == 0)]
    year_list = df1.groupby(['Development period'])['Underwriting YoA'].apply(list).to_dict()
    year_name_list= df1['Underwriting YoA'].unique()
    devprd_list = df1['Development period'].unique().tolist()
    development_factor_list, numerators, denominators = [],[],[]
    numerator = 0
    denominator = df1.loc[df1['Development period']==devprd_list[0],column_name].sum()
    numerators.append(np.nan)
    denominators.append(np.nan)
    for i, devprd in enumerate(devprd_list): 
        if devprd != devprd_list[0]:
            numerator = df1.loc[df1['Development period']==devprd,column_name].sum()
            if year_list[devprd] !=  year_list[devprd_list[i-1]]:
                common_years = set(year_list[devprd]) & set(year_list[devprd_list[i-1]])
                numerator = denominator = 0
                for j in common_years:
                    numerator = numerator + df1.loc[((df1['Development period']==devprd) &(df1['Underwriting YoA']==j))][column_name].tolist()[0]
                    denominator = denominator + df1.loc[((df1['Development period']==devprd_list[i-1]) &(df1['Underwriting YoA']==j))][column_name].tolist()[0]
            development_factor_list.append(numerator/denominator)
            numerators.append(numerator)
            denominators.append(denominator)
            denominator = numerator
    development_factor_list.insert(0, np.nan)
    df1new = pd.DataFrame()
    df1new['Development period'] = devprd_list
    df1new['Number of data points'] = [len(k) for k in year_list.values()]
    df1new['Numerator'] = numerators
    df1new['Denominator'] = denominators
    df1new['Development factor'] = development_factor_list

    listOfParams = devPcntWithPick(df,column_name, df1, df1new , year_name_list, piv_table, prem_piv_table, devprd_list)
    return listOfParams

# adds a new column to our dataframe, and asks for user input for picking percentages
def devPcntWithPick(df, column_name, df1, df1new, year_name_list, piv_table, prem_piv_table, devprd_list):    
    overall_devprod = df1new['Development factor'].product()
    dev_pcnt = df1new['Development factor'].cumprod().div(overall_devprod).tolist()
    df1new['Development percentages'] = dev_pcnt    
    dev_pcnt_with_pick = df1new['Development percentages'].copy().tolist()
    df1new['Development percentages with pick'] = dev_pcnt_with_pick
    validPickNum = False
    while validPickNum == False: 
        pickNum = input("Pick the index including which you would like to be 100% for " + column_name+ ": " )
        if (pickNum.upper() == 'NONE'):
            validPickNum= True
        elif pickNum.isnumeric():
            if (int(pickNum) < len(df1new)) & (int(pickNum) >= 0):
                validPickNum = True
    if pickNum.upper() != 'NONE':                       
        pickNum = int(pickNum)
        df1new.loc[pickNum:, 'Development percentages with pick'] = 1
        # this only needs to be done for gross premiums received
        if 'Gross Premiums Received' in column_name:
            df1new['temp cumulative product'] = df1new['Development factor'].cumprod().tolist()
            df1new['Development percentages with pick'] = df1new['Development factor'].cumprod().div(df1new['temp cumulative product'][pickNum])
            df1new.drop(['temp cumulative product'], axis = 1, inplace =True) 
            df1new.loc[pickNum:, 'Development percentages with pick'] = 1
        df1new.loc[0,'Development percentages with pick'] =  df1new.loc[1,'Development percentages']/df1new.loc[1,'Development factor']
    listOfParams = createNewTable(df, column_name, df1, df1new, year_name_list, piv_table, prem_piv_table, devprd_list)
    return listOfParams

#creates the new table that contains some ultimate information (latest development period info)
def createNewTable(df, column_name, df1, df1new, year_name_list, piv_table, prem_piv_table, devprd_list):
    df_latestDevPrd = pd.DataFrame()
    df_latestDevPrd['Underwriting YoA'] = year_name_list
    latest_dev_prd = piv_table.apply(pd.Series.last_valid_index, axis=1).tolist()
    df_latestDevPrd['Latest development period'] = latest_dev_prd
    df_latestDevPrd[column_name] = piv_table.apply(getLastNonNA, axis=1).tolist()
    devPcnt = []
    for i in latest_dev_prd:
        devPcnt.append(float(df1new.loc[((df1new['Development period']==i))]['Development percentages with pick'].tolist()[0]))
    df_latestDevPrd['% ' + column_name +  ' developed'] = devPcnt
    df_latestDevPrd['Ultimate ' + column_name] = df_latestDevPrd[column_name] / df_latestDevPrd['% '+ column_name +' developed']
    if 'Gross Incurred Claims' in column_name:
        df_latestDevPrd = appendExtraIncurredColumns(df, column_name,piv_table, df1, df1new, df_latestDevPrd, prem_piv_table, devprd_list, devPcnt)
    listOfParams = ultimateTable(df, column_name,piv_table, df1, df1new, df_latestDevPrd, prem_piv_table, devprd_list, devPcnt)
    return listOfParams

#adds on the extra columns if we are dealing with gross incurred claims. It also selects the correct Incurred Freultimate value 
#depending on the % incurred developed from pattern (CL ultimate if %>=75, BF Ultimate if 30<= % <70, Prior ultimate if % < 30)  
def appendExtraIncurredColumns(df, column_name,piv_table, df1, df1new, df_latestDevPrd, prem_piv_table, devprd_list, devPcnt):
    tempdf= pd.DataFrame()
    tempdf['Premiums Received'] = prem_piv_table.apply(getLastNonNA, axis=1).tolist().copy()
    tempdf['% premiums developed'] = devPcnt
    df_latestDevPrd['Ultimate premium'] =  tempdf['Premiums Received'] / tempdf['% premiums developed']
    df_latestDevPrd['CL ultimate'] = df_latestDevPrd['Gross Incurred Claims'] / df_latestDevPrd['% Gross Incurred Claims developed']
    df_latestDevPrd['Prior ultimate'] = tabledf['BF Prior / Initial LR'] * df_latestDevPrd['Ultimate premium']
    df_latestDevPrd['BF ultimate'] = (df_latestDevPrd['% Gross Incurred Claims developed']*df_latestDevPrd['CL ultimate'])+((1-df_latestDevPrd['% Gross Incurred Claims developed'])*df_latestDevPrd['Prior ultimate'])
    df_latestDevPrd['Incurred ultimate'] = df_latestDevPrd.apply(alterIncUlt, axis = 1) 
    return df_latestDevPrd

# choose the incurred ultimate value depending on the % inc developed
def alterIncUlt(x):
    if x['% Gross Incurred Claims developed']>=0.75:
        return x['CL ultimate']
    elif ((x['% Gross Incurred Claims developed']>=0.3)&(x['% Gross Incurred Claims developed']<0.75)):
        return x['BF ultimate'] 
    else:
        return x['Prior ultimate']
        
#creates the final pivot table that the expectations graph will ultimately be created from
def ultimateTable(df, column_name, piv_table, df1, df1new, df_latestDevPrd, prem_piv_table, devprd_list,devPcnt,):
    df3 = df1.copy()
    df_latestDevPrdCopy = pd.DataFrame()
    df_latestDevPrdCopy[['Underwriting YoA', 'Ultimate ' + column_name]] = df_latestDevPrd[['Underwriting YoA', 'Ultimate ' + column_name]].copy()
    df3 = df3.merge(df_latestDevPrdCopy, on = 'Underwriting YoA', how='left')
    df3['Percentage ' +column_name] = df3[column_name]/df3['Ultimate '+ column_name]
    df3.drop([column_name], axis =1, inplace =True)   
    pivdf3 = pd.pivot_table(df3, values ='Percentage ' +column_name , index = df3['Underwriting YoA'], columns = df3['Development period'])
    # passing parameters as a list so we can call this and the graph creation functions separately  
    listOfParams = [piv_table,column_name, pivdf3,df1new, devprd_list, devPcnt]
    return [piv_table,column_name, pivdf3,df1new, devprd_list, df_latestDevPrd, devPcnt]
    
#plots both of the graphs 
def plotBothGraphs(listOfParams):
    #unpacking the list of parameters
    piv_table = listOfParams[0]
    column_name = listOfParams[1]
    pivdf3 = listOfParams[2]
    df1new = listOfParams[3]
    devprd_list =listOfParams[4]
    
    fig1, axes = plt.subplots(1, 2)
    #setting a figure title
    fig1.suptitle('GRAPHS:')
    piv_table.transpose().plot(ax = axes[0]) 
    pivdf3.transpose().plot(ax = axes[1], linewidth = 0.75) 
    #plot the first graph
    axes[0].set(xlabel='Development period', ylabel=column_name, title=column_name)
    axes[0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(yaxisFormatting))
    #plot the expectations graph
    axes[1].set(xlabel='Development period')
    selected_pattern = df1new['Development percentages with pick'].tolist()
    plt.plot(devprd_list, selected_pattern, 'k--', label = 'Selected Pattern')
    axes[1].legend()
    axes[1].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(ypercentformat))
    plt.show()

# Calling the function

In [ ]:
plotEverything(df, 'Gross Premiums Received')
plotEverything(df, 'Gross Incurred Claims')
plotEverything(df, 'Gross Paid Claims')